# COVID-19 Projection

In [1]:
# library importing
import pandas as pd
import numpy as np
import nltk
import string
import seaborn as sns

In [5]:
us = pd.read_csv("../data/time_series_covid19_confirmed_US.csv")
gb = pd.read_csv("../data/time_series_covid19_confirmed_global.csv")